In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *

Get Available Stations ID List

In [3]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

146


In [4]:
def afficher_info_bouees_aleatoires(buoy_datas, prefix=None, df_wanted=None):
    """
    Affiche les informations de DataFrames (marine et/ou météo) de deux bouées choisies aléatoirement.
    """
    # Sélection de deux bouées au hasard
    sample_buoy_id_marine = random.choice(list(buoy_datas.keys()))
    sample_buoy_id_meteo = random.choice(list(buoy_datas.keys()))

    # Choix du préfixe (ex: "Cleaned")
    marine_key = f"{prefix} Marine" if prefix else "Marine"
    meteo_key = f"{prefix} Meteo" if prefix else "Meteo"

    sample_marine_df = buoy_datas[sample_buoy_id_marine].get(marine_key)
    sample_meteo_df = buoy_datas[sample_buoy_id_meteo].get(meteo_key)

    df_marine_name = "Marine DataFrame"
    df_meteo_name = "Météo DataFrame"

    if df_wanted is None or df_wanted.lower() == "marine":
        if sample_marine_df is not None and not sample_marine_df.empty:
            print(f"🌊 {df_marine_name} pour {sample_buoy_id_marine}:")
            print(sample_marine_df.info())
        else:
            print(f"⚠️ {df_marine_name} pour {sample_buoy_id_marine} : Aucune donnée")

    if df_wanted is None or df_wanted.lower() == "meteo":
        if sample_meteo_df is not None and not sample_meteo_df.empty:
            print(f"☁️ {df_meteo_name} pour {sample_buoy_id_meteo}:")
            print(sample_meteo_df.info())
        else:
            print(f"⚠️ {df_meteo_name} pour {sample_buoy_id_meteo} : Aucune donnée")

def display_buoys_missing_df_counts(buoy_datas, prefix=None, df_wanted=None):
    total = len(buoy_datas)
    marine_empty = 0
    meteo_empty = 0

    marine_key = f"{prefix} Marine" if prefix else "Marine"
    meteo_key = f"{prefix} Meteo" if prefix else "Meteo"

    for buoy_id, data in buoy_datas.items():
        df_marine = data.get(marine_key)
        df_meteo = data.get(meteo_key)

        if (df_wanted is None or df_wanted.lower() == "marine") and (df_marine is None or df_marine.empty):
            marine_empty += 1

        if (df_wanted is None or df_wanted.lower() == "meteo") and (df_meteo is None or df_meteo.empty):
            meteo_empty += 1

    if df_wanted is None or df_wanted.lower() == "marine":
        print(f"\n🌊 Nombre de bouées sans données '{marine_key}' : {marine_empty}/{total}")

    if df_wanted is None or df_wanted.lower() == "meteo":
        print(f"\n☁️ Nombre de bouées sans données '{meteo_key}' : {meteo_empty}/{total}")


Filter Buoys by Remarks

In [5]:
access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Parcours des lignes de la DataFrame
for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne
    
    # Extraction du nom de la station si un ")" est trouvé
    if ")" in station_Location:
        station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
    else:
        station_name = station_Location.strip()  # Si pas de ")", on garde toute la chaîne

    station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

    # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
    station_name = station_name.rstrip(" )")

    # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
    if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
        url = get_buoy_url(station_id)
        access_error_url_list.append(url)
    else:
        pass
# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

Avant Filtre: 146
Après Filtre: 43


Build Buoys_datas Dict

In [6]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key =next(iter(buoy_datas))
first_key
buoy_datas[first_key]


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : south bermuda
🆔 Station ID : 41049
✅ Coordonnées extraites : Latitude = 27.50N, Longitude = 62.27W
🌊 Water Depth : 5480 m
🌡️ Sea Temp Depth : 1.5
🌬️ Barometer Elevation : 2.7
💨 Anemometer 

{'station_zone': 'grays reef',
 'lat_buoy': '31.40N',
 'lon_buoy': '80.87W',
 'Water_depth': '16 m',
 'sea_temp_depth': '2',
 'Barometer_elevation': '2.4',
 'Anemometer_height': '3.8',
 'Air_temp_height': '3.4',
 'url': 'https://www.ndbc.noaa.gov/station_page.php?station=41008'}

Collecte de données marines et météos

In [7]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]
        df_marine['Datetime'] = df_marine['Datetime'].dt.tz_localize(None)

        buoy_datas[buoy_id]["Marine"] = df_marine

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"br_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_').replace(' ', '_').lower()

    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue
        
        rename_columns(df_meteo, {'date':'Datetime'})
        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        df_meteo['Datetime'] = df_meteo['Datetime'].dt.tz_localize(None)
    
        buoy_datas[buoy_id]["Meteo"] = df_meteo
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

# Retirer les bouées avec des DataFrames vides ou None
buoy_datas = {buoy_id: data for buoy_id, data in buoy_datas.items() 
              if "Marine" in data and data["Marine"] is not None and not data["Marine"].empty
              and "Meteo" in data and data["Meteo"] is not None and not data["Meteo"].empty}

# 🔚 Résumé final

print("\n📝 Résumé final :")
print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")

# Afficher la longueur du dictionnaire (nombre de bouées avec des données valides)
print(f"\n📊 Nombre de bouées avec des données valides : {len(buoy_datas)}")


🚀 Démarrage du processus de collecte des données...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

📊 station_zone : grays reef
🔄 Colonne 'date' renommée en 'Datetime'
✅ Colonnes renommées : {'date': 'Datetime'}

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !

📊 station_zone : ne st martin
🔄 Colonne 'date' renommée en 'Datetime'
✅ Colonnes

In [8]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/41

☁️ Nombre de bouées sans données 'Meteo' : 0/41


Data Enrichment with MetaDatas

In [9]:
list_not_include = ['lon_buoy', "lat_buoy", "url"]
for buoy_id, value in buoy_datas.items():
    print(f"\n🔍 Traitement de la Station ID: {buoy_id}")

    marine_df = buoy_datas[buoy_id]["Marine"]
    meteo_df = buoy_datas[buoy_id]["Meteo"]

    try:
        # Récupérer les métadonnées de la station
        buoy_metadata = get_station_metadata(buoy_id)
        parsed_data = parse_buoy_json(buoy_metadata)

        # Mise à jour du dictionnaire avec les métadonnées
        data = buoy_datas[buoy_id]
        data.update(parsed_data)
        
        # Ajouter les métadonnées comme nouvelles colonnes dans marine_df
        if marine_df is not None:
            marine_df["Station ID"] = str(buoy_id)
            for key, value in parsed_data.items():
                # Vérifier si la clé n'est pas dans la liste des exclusions
                if key not in list_not_include:
                    marine_df[key] = value
                    print(f"✅ Colonne '{key}' ajoutée au DataFrame de la station {buoy_id}")

    except Exception as e:
        print(f"❌ Erreur pour la station {buoy_id}: {e}")

# Vérification de l'ajout des colonnes en prenant un id au hasard
station_id = random.choice(list(buoy_datas.keys()))
marine_df = buoy_datas[station_id]["Marine"]

if marine_df is not None:
    print("\nColonnes ajoutées au DataFrame de la station", station_id)
    print(marine_df.columns)


🔍 Traitement de la Station ID: 41008

🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

✅ Colonne 'station_zone' ajoutée au DataFrame de la station 41008
✅ Colonne 'Water_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'sea_temp_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'Barometer_elevation' ajoutée au DataFrame de la station 41008
✅ Colonne 'Anemometer_height' ajoutée au DataFrame de la station 41008
✅ Colonne 'Air_temp_height' ajoutée au DataFrame de la station 41008

🔍 Traitement de la Station ID: 41044

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, 

In [10]:
df_marine.head()

,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,...,Datetime,Lat,Lon,Water_depth,Station ID,station_zone,sea_temp_depth,Barometer_elevation,Anemometer_height,Air_temp_height
0,290.0,5.1,6.2,NaN,NaN,NaN,NaN,1019.9,2.2,NaN,...,2025-04-08 21:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
1,270.0,6.2,6.7,NaN,NaN,NaN,NaN,1020.1,1.5,NaN,...,2025-04-08 20:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
2,280.0,6.7,10.8,NaN,NaN,NaN,NaN,1020.3,0.7,NaN,...,2025-04-08 19:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
3,270.0,7.7,8.8,NaN,NaN,NaN,NaN,1020.6,-0.1,NaN,...,2025-04-08 18:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
4,300.0,7.2,8.2,NaN,NaN,NaN,NaN,1020.7,-0.9,NaN,...,2025-04-08 17:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4


In [11]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/41

☁️ Nombre de bouées sans données 'Meteo' : 0/41


In [12]:
important_columns_oceanography = [
    'wind_direction',             
    'wind_speed',                 
    'wave_height',                   
    'pressure',                   
    'air_temperature',            
    'water_temperature',          
    'Datetime',
    'Lat',
    'Lon'                 
]

important_columns_meteorology = [
    'temperature_2m',             
    'relative_humidity_2m',       
    'dew_point_2m',               
    'precipitation',              
    'pressure_msl',               
    'cloud_cover',                
    'wind_speed_10m',             
    'Datetime'
]

Handle Null Values

In [13]:
stations_depart = len(buoy_datas)
ignored_buoys = {}  # Dictionary to track ignored buoys and their reasons

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Nettoyage des données pour la station {station_id}")

    marine_df = data.get("Marine")
    meteo_df = data.get("Meteo")

    if marine_df is None or meteo_df is None:
        ignored_buoys[station_id] = "Marine DataFrame ou Meteo DataFrame manquant(e)"
        print(f"⚠️ Station {station_id} ignorée: Marine DataFrame ou Meteo DataFrame manquant(e)")
        continue

    try:
        # Nettoyage des DataFrames
        cleaned_marine_df = handle_null_values(marine_df)
        cleaned_meteo_df = handle_null_values(meteo_df)
        # Vérification des colonnes importantes après nettoyage
        marine_columns_ok = all(col in cleaned_marine_df.columns for col in important_columns_oceanography)
        meteo_columns_ok = all(col in cleaned_meteo_df.columns for col in important_columns_meteorology)

        # Track which columns are missing
        missing_marine_columns = [col for col in important_columns_oceanography if col not in cleaned_marine_df.columns]
        missing_meteo_columns = [col for col in important_columns_meteorology if col not in cleaned_meteo_df.columns]

        if missing_marine_columns or missing_meteo_columns:
            ignored_buoys[station_id] = f"Colonnes manquantes: Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}"
            print(f"⚠️ Station {station_id} ignorée: Colonnes manquantes - Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}")
            continue

        # Ajouter le DataFrame nettoyé au dictionnaire des résultats
        buoy_datas[station_id]['Cleaned Marine'] = cleaned_marine_df
        buoy_datas[station_id]['Cleaned Meteo'] = cleaned_meteo_df
        print(f"✅ Nettoyage réussi pour la station {station_id} ({cleaned_marine_df.shape[0]} lignes)")

    except Exception as e:
        ignored_buoys[station_id] = f"Erreur lors du nettoyage: {e}"
        print(f"❌ Erreur lors du nettoyage pour {station_id}: {e}")

# 🔥 Suppression des stations ignorées du dictionnaire principal
for station_id in ignored_buoys:
    buoy_datas.pop(station_id, None)

len_cleaned_data = len([data for data in buoy_datas.values() if 'Cleaned Marine' in data and 'Cleaned Meteo' in data])

# Résumé final du nettoyage
print("\n📊 RÉSUMÉ DU NETTOYAGE:")
print(f"📌 Stations au départ : {stations_depart}")
print(f"✅ Stations nettoyées : {len_cleaned_data}")
print(f"🏁 Stations restantes après filtrage :")

for station_id, reason in ignored_buoys.items():
    print(f"🛑 Station {station_id} ignorée: {reason}")

print(f"\n🧹 Clés restantes dans buoy_datas après purge : {len(buoy_datas)} (attendu : {len_cleaned_data})")


🔄 Nettoyage des données pour la station 41008

Tag: orange - Nombre de lignes: 6566
Colonne 'wind_direction' Imputée par la médiane (26.18% de valeurs manquantes)
Colonne 'wind_speed' Imputée par la médiane (25.92% de valeurs manquantes)
Colonne 'wind_gust' Imputée par la médiane (25.92% de valeurs manquantes)
Colonne 'wave_height' Imputée par la médiane (49.04% de valeurs manquantes)
Colonne 'dominant_wave_period' Supprimée (plus de 55% de valeurs manquantes)
Colonne 'average_wave_period' Imputée par la médiane (49.06% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (49.09% de valeurs manquantes)
Colonne 'pressure' Imputée par la médiane (0.17% de valeurs manquantes)
Colonne 'air_temperature' Imputée par la médiane (25.92% de valeurs manquantes)
Colonne 'water_temperature' Imputée par la médiane (16.57% de valeurs manquantes)
Colonne 'dewpoint' Imputée par la médiane (26.27% de valeurs manquantes)
Colonne 'visibility' Supprimée (plus de 55% de valeurs 

In [14]:
display_buoys_missing_df_counts(buoy_datas, prefix="Cleaned")


🌊 Nombre de bouées sans données 'Cleaned Marine' : 0/24

☁️ Nombre de bouées sans données 'Cleaned Meteo' : 0/24


In [15]:
# Fusion des DataFrames nettoyés
print("\n🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION")

merged_success_count = 0  # Compteur de fusions réussies
total_merged_rows = 0     # Total de lignes fusionnées

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Fusion des données pour la station {station_id}")

    cleaned_marine_df = data.get("Cleaned Marine")
    cleaned_meteo_df = data.get("Cleaned Meteo")

    if cleaned_marine_df is None or cleaned_meteo_df is None:
        continue

    try:
        merged_df = pd.merge(cleaned_marine_df, cleaned_meteo_df, on="Datetime", how="inner")

        if merged_df.empty:
            print(f"⚠️ Station {station_id} fusionnée, mais résultat vide après inner merge sur 'Datetime'")
        else:
            buoy_datas[station_id]["Merged"] = merged_df
            merged_success_count += 1
            total_merged_rows += len(merged_df)
            print(f"✅ Fusion réussie pour la station {station_id} ({merged_df.shape[0]} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la fusion pour {station_id}: {e}")

# Résumé des fusions
print(f"\n📦 Fusions réussies : {merged_success_count}/{len_cleaned_data} stations")
print(f"📊 Total de lignes fusionnées : {total_merged_rows}")


🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION

🔄 Fusion des données pour la station 41008
✅ Fusion réussie pour la station 41008 (1101 lignes)

🔄 Fusion des données pour la station 41044
✅ Fusion réussie pour la station 41044 (1101 lignes)

🔄 Fusion des données pour la station 41049
✅ Fusion réussie pour la station 41049 (1101 lignes)

🔄 Fusion des données pour la station 42001
✅ Fusion réussie pour la station 42001 (640 lignes)

🔄 Fusion des données pour la station 42002
✅ Fusion réussie pour la station 42002 (664 lignes)

🔄 Fusion des données pour la station 42036
✅ Fusion réussie pour la station 42036 (1092 lignes)

🔄 Fusion des données pour la station 42056
✅ Fusion réussie pour la station 42056 (1101 lignes)

🔄 Fusion des données pour la station 42058
✅ Fusion réussie pour la station 42058 (1098 lignes)

🔄 Fusion des données pour la station 44007
✅ Fusion réussie pour la station 44007 (1100 lignes)

🔄 Fusion des données pour la station 44020
✅ Fusion réussie pour la station 440

In [16]:
# Concaténation des DataFrames fusionnés
print("\n🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME")

final_merged_df_list = []
concat_success_count = 0
concat_total_rows = 0

for station_id, data in buoy_datas.items():
    merged_df = data.get("Merged")

    if merged_df is None:
        print(f"⚠️ Station {station_id} ignorée pour concaténation: Données fusionnées manquantes")
        continue

    try:
        final_merged_df_list.append(merged_df)
        concat_success_count += 1
        concat_total_rows += len(merged_df)
        print(f"✅ Concaténation réussie pour la station {station_id} ({len(merged_df)} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la concaténation pour {station_id}: {e}")

# Création du DataFrame final unique
try:
    df_final = pd.concat(final_merged_df_list, ignore_index=True)
    print(f"\n🧾 DataFrame final créé avec succès ({df_final.shape[0]} lignes, {df_final.shape[1]} colonnes)")
except Exception as e:
    print(f"\n❌ Erreur lors de la création du DataFrame final: {e}")
    df_final = None

# Résumé
print(f"\n📦 Concaténations réussies : {concat_success_count}/{merged_success_count}")
print(f"📊 Total de lignes dans le DataFrame final : {concat_total_rows}")


🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME
✅ Concaténation réussie pour la station 41008 (1101 lignes)
✅ Concaténation réussie pour la station 41044 (1101 lignes)
✅ Concaténation réussie pour la station 41049 (1101 lignes)
✅ Concaténation réussie pour la station 42001 (640 lignes)
✅ Concaténation réussie pour la station 42002 (664 lignes)
✅ Concaténation réussie pour la station 42036 (1092 lignes)
✅ Concaténation réussie pour la station 42056 (1101 lignes)
✅ Concaténation réussie pour la station 42058 (1098 lignes)
✅ Concaténation réussie pour la station 44007 (1100 lignes)
✅ Concaténation réussie pour la station 44020 (1100 lignes)
✅ Concaténation réussie pour la station 44065 (1102 lignes)
✅ Concaténation réussie pour la station 46006 (1098 lignes)
✅ Concaténation réussie pour la station 46014 (1101 lignes)
✅ Concaténation réussie pour la station 46025 (1100 lignes)
✅ Concaténation réussie pour la station 46027 (1099 lignes)
✅ Concaténation réussie pour la station 4

In [17]:
display(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25503 entries, 0 to 25502
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   wind_direction           25503 non-null  float64       
 1   wind_speed               25503 non-null  float64       
 2   wind_gust                25503 non-null  float64       
 3   wave_height              25503 non-null  float64       
 4   average_wave_period      25503 non-null  float64       
 5   dominant_wave_direction  24403 non-null  float64       
 6   pressure                 25503 non-null  float64       
 7   air_temperature          25503 non-null  float64       
 8   water_temperature        25503 non-null  float64       
 9   dewpoint                 25503 non-null  float64       
 10  Datetime                 25503 non-null  datetime64[ns]
 11  Lat                      25503 non-null  object        
 12  Lon                      25503 n

None

Hour Filtering

In [18]:
try:
    df_final['Datetime'] = pd.to_datetime(df_final['Datetime'])

    df_final = df_final[df_final['Datetime'].dt.minute == 0]
    # Affichage pour vérifier le résultat
    print(f"🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: {df_final.shape[0]} lignes")

except Exception as e:
    print(f"�� Erreur lors du filtrage des lignes à l'heure pile: {e}")

🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: 25503 lignes


In [19]:
final_datetime = pd.DataFrame(df_final[['Datetime']], dtype='datetime64[ns]')
final_datetime

,Datetime
0,2025-04-08 21:00:00
1,2025-04-08 20:00:00
2,2025-04-08 19:00:00
3,2025-04-08 18:00:00
4,2025-04-08 17:00:00
...,...
25498,2025-02-22 04:00:00
25499,2025-02-22 03:00:00
25500,2025-02-22 02:00:00
25501,2025-02-22 01:00:00


In [20]:
df_final = handle_null_values(df_final)


Tag: yellow - Nombre de lignes: 25503
Colonne 'wind_direction' non modifiée (0% de valeurs manquantes)
Colonne 'wind_speed' non modifiée (0% de valeurs manquantes)
Colonne 'wind_gust' non modifiée (0% de valeurs manquantes)
Colonne 'wave_height' non modifiée (0% de valeurs manquantes)
Colonne 'average_wave_period' non modifiée (0% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (4.31% de valeurs manquantes)
Colonne 'pressure' non modifiée (0% de valeurs manquantes)
Colonne 'air_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'water_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'dewpoint' non modifiée (0% de valeurs manquantes)
Colonne 'Datetime' non modifiée (0% de valeurs manquantes)
Colonne 'Lat' non modifiée (0% de valeurs manquantes)
Colonne 'Lon' non modifiée (0% de valeurs manquantes)
Colonne 'Water_depth' non modifiée (0% de valeurs manquantes)
Colonne 'Station ID' non modifiée (0% de valeurs manquantes)
Colonne 'stati

In [21]:
display_row_values(df_final)

wind_direction  |  wind_speed  |  wind_gust  |  wave_height  |  average_wave_period  |  dominant_wave_direction  |  pressure  |  air_temperature  |  water_temperature  |  dewpoint  |  Datetime             |  Lat     |  Lon      |  Water_depth  |  Station ID  |  station_zone                              |  sea_temp_depth  |  Barometer_elevation  |  Anemometer_height  |  Air_temp_height  |  temperature_2m  |  relative_humidity_2m  |  dew_point_2m  |  precipitation  |  rain  |  showers  |  pressure_msl  |  surface_pressure  |  cloud_cover  |  cloud_cover_low  |  cloud_cover_mid  |  cloud_cover_high  |  visibility  |  wind_speed_10m  |  soil_temperature_0cm  |  soil_moisture_0_to_1cm  |  is_day
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
# get station id distinct values
station_ids = df_final['Station ID'].unique()
print(station_ids)

['41008' '41044' '41049' '42001' '42002' '42036' '42056' '42058' '44007'
 '44020' '44065' '46006' '46014' '46025' '46027' '46072' '46078' '46084'
 '46086' '46087' '46088' '51000' '51001' '51002']


In [23]:
df_final.columns = [col.strip() for col in df_final.columns]

In [24]:
def display_row_values(df, columns=None):
    # Si aucune colonne n'est spécifiée, afficher toutes les colonnes
    if columns is None:
        columns = df.columns.tolist()

    if isinstance(columns, str):
        columns = [columns]
    # Calculer la largeur maximale pour chaque colonne spécifiée
    column_widths = [max(df[col].astype(str).apply(len).max(), len(col)) for col in columns]

    # Afficher les noms des colonnes, en tenant compte des largeurs maximales
    column_headers = [col.ljust(column_widths[i]) for i, col in enumerate(columns)]
    print("  |  ".join(column_headers))
    print("-" * (sum(column_widths) + (len(columns) - 1) * 4))  # Ligne de séparation

    # Afficher les 10 premières valeurs sous chaque colonne spécifiée, alignées
    for i in range(min(10, len(df))):  # Affiche jusqu'à 10 lignes ou le nombre de lignes disponibles
        row_values = [str(df.iloc[i, df.columns.get_loc(col)]).ljust(column_widths[j]) for j, col in enumerate(columns)]
        print("  |  ".join(row_values))

In [25]:
# Dictionnaire de renommage des colonnes
col_to_rename = {'temperature_2m': 'T°(C°)', 
                 'relative_humidity_2m': 'Relative Humidity (%)',
                 'dew_point_2m': 'Dew Point (°C)', 
                 'precipitation': 'Precipitations (mm)',  
                 'pressure_msl':'Sea Level Pressure (hPa)', 
                 'cloud_cover_low':'Low Clouds (%)',
                 'cloud_cover_mid' : 'Middle Clouds (%)', 
                 'cloud_cover_high' : 'High Clouds (%)', 
                 'visibility' : 'Visibility (km)', 
                 'wind_direction': 'Wind Direction (°)',
                 'wind_speed': 'Wind Speed (km/h)', 
                 'wind_gust': 'Wind Gusts (km/h)',
                 'wind_speed_10m':'Wind Speed (10m)', 
                 'surface_pressure': 'Surface Pressure',
                 'wave_height': 'Wave Height (m)', 
                 'average_wave_period': 'Average Wave Period (s)',
                 'dominant_wave_direction': 'Dominant Wave Direction (°)', 
                 'pressure': 'Pressure (hPa)',
                 'air_temperature': 'Air T°', 
                 'water_temperature': 'Water T°', 
                 'Water_depth': 'Water Depth (m)', 
                 "Air_temp_height": "Air T° Height", 
                 "Anemometer_height": "Anemometer Height (m)", 
                 "station_zone": "Station Zone",
                 "Barometer_elevation": "Barometer Elevation", 
                 "sea_temp_depth" : "Sea Temperature Depth (m)"}

# Liste des colonnes à supprimer
cols_to_delete = ['soil_temperature_0cm', 'lat_buoy','lon_buoy', 'rain', 
                  'showers', 'is_day', 'soil_moisture_0_to_1cm', 
                  'cloud_cover']
	

# Renommer les colonnes d'abord
df_final = rename_columns(df_final, col_to_rename)
# Ensuite, supprimer les colonnes non désirées
df_final = drop_columns_if_exist(df_final, cols_to_delete)
try:
    df_final["Visibility (km)"].map(lambda x: x/1000)
    df_final["T°(C°)"] = round(df_final["T°(C°)"], 2)
    df_final["Wind Speed (10m)"] = round(df_final["Wind Speed (10m)"], 2)
except Exception as e:
    print(f"�� Erreur lors du traitement des colonnes :\n {e}")

# Afficher les résultats
print("\nColonnes après renommage et suppression :")
print(df_final.columns)

🔄 Colonne 'temperature_2m' renommée en 'T°(C°)'
🔄 Colonne 'relative_humidity_2m' renommée en 'Relative Humidity (%)'
🔄 Colonne 'dew_point_2m' renommée en 'Dew Point (°C)'
🔄 Colonne 'precipitation' renommée en 'Precipitations (mm)'
🔄 Colonne 'pressure_msl' renommée en 'Sea Level Pressure (hPa)'
🔄 Colonne 'cloud_cover_low' renommée en 'Low Clouds (%)'
🔄 Colonne 'cloud_cover_mid' renommée en 'Middle Clouds (%)'
🔄 Colonne 'cloud_cover_high' renommée en 'High Clouds (%)'
🔄 Colonne 'visibility' renommée en 'Visibility (km)'
🔄 Colonne 'wind_direction' renommée en 'Wind Direction (°)'
🔄 Colonne 'wind_speed' renommée en 'Wind Speed (km/h)'
🔄 Colonne 'wind_gust' renommée en 'Wind Gusts (km/h)'
🔄 Colonne 'wind_speed_10m' renommée en 'Wind Speed (10m)'
🔄 Colonne 'surface_pressure' renommée en 'Surface Pressure'
🔄 Colonne 'wave_height' renommée en 'Wave Height (m)'
🔄 Colonne 'average_wave_period' renommée en 'Average Wave Period (s)'
🔄 Colonne 'dominant_wave_direction' renommée en 'Dominant Wave Di

In [26]:
try:
    display_row_values(df_final, columns=["Visibility (km)", "T°(C°)"])
#    df_final["Visibility (km)"] = df_final["Visibility (km)"].map(lambda x: x / 1000)
    
    df_final["T°(C°)"] = df_final["T°(C°)"].round(2)
    display_row_values(df_final, ["Visibility (km)", "T°(C°)"])
except Exception as e:
    print(f"�� Erreur : {e}")

Visibility (km)  |  T°(C°)
-------------------------
26400.0          |  17.9  
23900.0          |  17.5  
20500.0          |  17.2  
18800.0          |  16.3  
17000.0          |  15.7  
17900.0          |  15.35 
16500.0          |  14.8  
18000.0          |  15.25 
15600.0          |  15.15 
18900.0          |  16.0  
Visibility (km)  |  T°(C°)
-------------------------
26400.0          |  17.9  
23900.0          |  17.5  
20500.0          |  17.2  
18800.0          |  16.3  
17000.0          |  15.7  
17900.0          |  15.35 
16500.0          |  14.8  
18000.0          |  15.25 
15600.0          |  15.15 
18900.0          |  16.0  


In [27]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Air T°  |  Water T°  |  dewpoint  |  Datetime             |  Lat     |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Surface Pressure  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
try:
    df_final[['Lat', 'Lon']] = df_final.apply(
        lambda row: pd.Series(convert_coordinates(row['Lat'], row['Lon'])),
        axis=1
    )
except Exception as e:
    print(f"❌ Erreur lors de la conversion des coordonnées : {e}")


In [29]:
#  T°(C°) = (Air T° + T°(C°))/2 et virer Air T°
try:
    df_final['T°(C°)'] = (df_final['Air T°'] + df_final['T°(C°)']) / 2
    df_final.drop(columns=['Air T°'], inplace=True)
except:
    pass

# Sea Level Pressure (hPa) = (Sea Level Pressure (hPa) +Surface Pressure)/2 et virer Surface Pressure

try:
    df_final['Sea Level Pressure (hPa)'] = (df_final['Sea Level Pressure (hPa)'] + df_final['Surface Pressure']) / 2
    df_final.drop(columns=['Surface Pressure'], inplace=True)
except:
    pass

# virer le m dans Water Depth avec regex lambda et passer la colonne en float
df_final['Water Depth (m)'] = df_final['Water Depth (m)'].apply(lambda x: re.sub(r'\D', '', str(x)).strip())
df_final['Water Depth (m)'].astype(float)

0          16.0
1          16.0
2          16.0
3          16.0
4          16.0
          ...  
25498    4979.0
25499    4979.0
25500    4979.0
25501    4979.0
25502    4979.0
Name: Water Depth (m), Length: 25503, dtype: float64

In [30]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Water T°  |  dewpoint  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Check truth about Wind Speed Using another API call

In [31]:
df_42058 = df_final[df_final['Station ID'] == "42058"]
df_42058.columns

Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPa)', 'Water T°', 'dewpoint',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Anemometer Height (m)', 'Air T° Height', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)',
       'High Clouds (%)', 'Visibility (km)', 'Wind Speed (10m)'],
      dtype='object')

In [32]:
# ---- Chargement de la clé API ----
vc_api_key_path = r"c:\Credentials\visual_crossing_weather_api.json"
with open(vc_api_key_path, 'r') as file:
    content = json.load(file)
    vc_api_key = content["api_key"]

# ---- Extraire les coordonnées depuis la première ligne du DataFrame ----
lat_42058, lon_42058 = None, None
if not df_42058.empty:
    first_row = df_42058.iloc[0]
    lat_42058, lon_42058 = first_row["Lat"], first_row["Lon"]

# ---- Définir les dates pour la requête ----
today = datetime.now().strftime("%Y-%m-%d")
last_month = (datetime.now() - timedelta(days=31)).strftime("%Y-%m-%d")

# ---- Créer le dossier de cache si nécessaire ----
cache_dir = "api_call_files"
os.makedirs(cache_dir, exist_ok=True)

# ---- Définir le fichier cache selon la position ----
cache_file = os.path.join(cache_dir, f"vc_meteo_{lat_42058}_{lon_42058}.csv")

# ---- Vérifier si un cache récent existe (moins de 24h) ----
use_cache = False
if os.path.exists(cache_file):
    last_modified = datetime.fromtimestamp(os.path.getmtime(cache_file))
    if datetime.now() - last_modified < timedelta(hours=24):
        print(f"📦 Cache détecté ({cache_file}), modifié le {last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
        vc_meteo_df = pd.read_csv(cache_file)
        print("✅ Données météo rechargées depuis le cache.")
        use_cache = True
    else:
        print(f"⚠️ Cache trouvé mais périmé (plus de 24h) → nouvelle requête API.")

# ---- Appel API si pas de cache valide ----
if not use_cache and lat_42058 is not None and lon_42058 is not None:
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat_42058},{lon_42058}/{last_month}/{today}?unitGroup=metric&key={vc_api_key}&contentType=json"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            vc_meteo_data = response.json()
            print("🌍 Données météo récupérées depuis l'API Visual Crossing.")

            # ---- Extraire les données journalières et sauvegarder ----
            if "days" in vc_meteo_data:
                vc_meteo_df = pd.json_normalize(vc_meteo_data["days"])
                vc_meteo_df.to_csv(cache_file, index=False)
                print(f"💾 Données sauvegardées dans le cache : {cache_file}")
            else:
                print("⚠️ Le champ 'days' est absent de la réponse API.")
        else:
            print(f"❌ Échec de l’appel API — code de statut : {response.status_code}")
    except Exception as e:
        print(f"❌ Exception levée lors de la requête API : {e}")

📦 Cache détecté (api_call_files\vc_meteo_14.51_-75.15.csv), modifié le 2025-04-08 18:00:42
✅ Données météo rechargées depuis le cache.


In [33]:
# 📁 Charger les données du CSV Visual Crossing
vc_csv_path = f"api_call_files/vc_meteo_{lat_42058}_{lon_42058}.csv"
df_vc_meteo = pd.read_csv(vc_csv_path)
df_vc_meteo.head()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,hours
0,2025-03-08,1741410000,27.4,26.9,27.2,30.7,29.3,30.1,23.5,80.1,...,1741432362,18:10:05,1741475405,0.30,Rain,Clear conditions throughout the day with rain ...,rain,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
1,2025-03-09,1741496400,27.5,26.9,27.3,30.9,29.6,30.4,23.8,81.3,...,1741518723,18:10:14,1741561814,0.34,Rain,Clear conditions throughout the day with morni...,rain,"['42058_maritime', 'remote']",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
2,2025-03-10,1741582800,27.4,26.9,27.3,30.3,29.3,30.0,23.2,78.5,...,1741605083,18:10:23,1741648223,0.37,Clear,Clear conditions throughout the day.,clear-day,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
3,2025-03-11,1741669200,27.4,26.9,27.2,30.2,29.1,29.7,22.9,77.5,...,1741691442,18:10:31,1741734631,0.41,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
4,2025-03-12,1741755600,27.3,26.5,27.0,29.9,26.5,28.9,22.4,76.1,...,1741777802,18:10:40,1741821040,0.44,Clear,Clear conditions throughout the day.,clear-day,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."


In [34]:
# 🧼 Nettoyage et transformation
import ast  # Permet de convertir les strings représentant des listes en objets Python
all_hours = []

# Parcours de chaque ligne du fichier météo
for i, row in df_vc_meteo.iterrows():
    try:
        # Convertir la colonne "hours" en liste de dictionnaires
        hours_list = ast.literal_eval(row['hours'])

        # Pour chaque heure dans la journée, on ajoute une entrée au tableau final
        for hour_data in hours_list:
            hour_data['Date'] = row['datetime']  # On ajoute aussi la date du jour correspondant
            all_hours.append(hour_data)

    except Exception as e:
        print(f"Erreur parsing ligne {i}: {e}")  # En cas d'erreur de parsing (souvent problème de format)

# ✅ On transforme la liste aplanie en DataFrame
df_vc_flat = pd.DataFrame(all_hours)

# 🕒 Conversion de l'heure/temps
df_vc_flat["Datetime"] = pd.to_datetime(df_vc_flat["datetimeEpoch"], unit="s").dt.strftime("%Y-%m-%d-%H")

# 🗓️ Filtrer sur les 30 derniers jours
today = datetime.now()
thirty_days_ago = today - timedelta(days=30)

today_str = today.strftime("%Y-%m-%d")
thirty_days_ago_str = thirty_days_ago.strftime("%Y-%m-%d")

# On filtre le DataFrame pour ne garder que les lignes entre ces deux dates
df_vc_last_month = df_vc_flat[
    (df_vc_flat['Date'] >= thirty_days_ago_str) & 
    (df_vc_flat['Date'] <= today_str)]

In [35]:
display_row_values(df_vc_last_month)

datetime  |  datetimeEpoch  |  temp  |  feelslike  |  humidity  |  dew   |  precip  |  precipprob  |  snow  |  snowdepth  |  preciptype  |  windgust  |  windspeed  |  winddir  |  pressure  |  visibility  |  cloudcover  |  solarradiation  |  solarenergy  |  uvindex  |  severerisk  |  conditions              |  icon                 |  stations            |  source  |  Date        |  Datetime     
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
00:00:00  |  1741496400     |  27.2  |  30.3       |  81.71     |  23.8  |  0.0     |  0.0         |  0.0   |  0.0        |  None        |  38.2      |  28.8       |  80.0     |  1013.6    |  24.1        |  6.8         |  0.0     

In [36]:
# Correct syntaxe pour filtrer les colonnes spécifiques
df_vc_last_month = df_vc_last_month[["Datetime", "temp", "humidity", "precip", "dew", "windgust", "windspeed", "winddir", "pressure", "visibility"]]

for col in df_vc_last_month.columns:
    if not col == "Datetime":
        rename_columns(df_vc_last_month, {col: f"VC_{col}"})

🔄 Colonne 'temp' renommée en 'VC_temp'
✅ Colonnes renommées : {'temp': 'VC_temp'}
🔄 Colonne 'humidity' renommée en 'VC_humidity'
✅ Colonnes renommées : {'humidity': 'VC_humidity'}
🔄 Colonne 'precip' renommée en 'VC_precip'
✅ Colonnes renommées : {'precip': 'VC_precip'}
🔄 Colonne 'dew' renommée en 'VC_dew'
✅ Colonnes renommées : {'dew': 'VC_dew'}
🔄 Colonne 'windgust' renommée en 'VC_windgust'
✅ Colonnes renommées : {'windgust': 'VC_windgust'}
🔄 Colonne 'windspeed' renommée en 'VC_windspeed'
✅ Colonnes renommées : {'windspeed': 'VC_windspeed'}
🔄 Colonne 'winddir' renommée en 'VC_winddir'
✅ Colonnes renommées : {'winddir': 'VC_winddir'}
🔄 Colonne 'pressure' renommée en 'VC_pressure'
✅ Colonnes renommées : {'pressure': 'VC_pressure'}
🔄 Colonne 'visibility' renommée en 'VC_visibility'
✅ Colonnes renommées : {'visibility': 'VC_visibility'}


In [37]:
# Convert 'Datetime' column to datetime type in df_vc_last_month
df_vc_last_month['Datetime'] = pd.to_datetime(df_vc_last_month['Datetime'], errors='coerce')


# Check if the Datetime column is correctly converted
print(df_42058['Datetime'].dtype)
print(df_vc_last_month['Datetime'].dtype)

datetime64[ns]
datetime64[ns]


In [38]:
# Now perform the merge
df_compare = pd.merge(df_42058, df_vc_last_month, on="Datetime", how="inner")

df_compare.columns

Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPa)', 'Water T°', 'dewpoint',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Anemometer Height (m)', 'Air T° Height', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)',
       'High Clouds (%)', 'Visibility (km)', 'Wind Speed (10m)', 'VC_temp',
       'VC_humidity', 'VC_precip', 'VC_dew', 'VC_windgust', 'VC_windspeed',
       'VC_winddir', 'VC_pressure', 'VC_visibility'],
      dtype='object')

In [39]:
try:
    df_windspeed_compare = df_compare[['Wind Speed (km/h)', 'Wind Speed (10m)','VC_windspeed']]

    df_pressure_compare = df_compare[['Sea Level Pressure (hPa)','Pressure (hPa)', 'VC_pressure']]
    df_dew_compare = df_compare[['dewpoint','Dew Point (°C)', 'VC_dew']]
except Exception as e:
    print(e)

Comparaison Wind Speed

In [40]:
df_windspeed_compare.head(10)

,Wind Speed (km/h),Wind Speed (10m),VC_windspeed
0,7.0,25.959999,31.3
1,7.0,26.280001,31.7
2,8.0,27.799999,31.3
3,8.0,29.530001,30.6
4,8.0,30.549999,30.6
5,9.0,31.930000,31.0
6,9.0,33.279999,33.1
7,9.0,33.509998,32.4
8,9.0,34.220001,32.4
9,10.0,35.310001,36.0


In [41]:
wind_col_to_delete = ['Wind Speed (km/h)', 'Anemometer Height (m)']
df_final = drop_columns_if_exist(df_final, wind_col_to_delete)

 Nombre initial de colonnes: 29
Colonne 'Wind Speed (km/h)' Supprimée
Colonne 'Anemometer Height (m)' Supprimée
 Nombre final de colonnes: 27


Comparaison Pressure

In [42]:
df_pressure_compare.head()

,Sea Level Pressure (hPa),Pressure (hPa),VC_pressure
0,1010.200012,1009.5,1010.0
1,1010.400024,1009.6,1010.0
2,1010.799988,1010.2,1011.0
3,1011.700012,1010.8,1012.0
4,1012.200012,1011.5,1012.0


In [43]:
df_final = drop_columns_if_exist(df_final, ['Pressure (hPa)'])

 Nombre initial de colonnes: 27
Colonne 'Pressure (hPa)' Supprimée
 Nombre final de colonnes: 26


In [44]:
df_dew_compare.head(20)

,dewpoint,Dew Point (°C),VC_dew
0,24.3,23.010000,23.4
1,24.0,23.059999,23.1
2,23.9,23.309999,23.1
3,23.8,23.350000,23.2
4,24.1,22.840000,23.3
5,23.8,22.910000,23.3
6,24.0,22.770000,23.2
7,23.7,22.620001,23.8
8,23.7,22.480000,23.8
9,23.3,22.430000,23.4


In [45]:
try:

    # Calcul des distances absolues entre chaque paire de mesures pour chaque ligne
    # Ces distances nous permettent de savoir quelle mesure est la plus proche des autres
    df_compare['dist_dewpoint_DewPoint'] = np.abs(df_compare['dewpoint'] - df_compare['Dew Point (°C)'])
    df_compare['dist_dewpoint_VC'] = np.abs(df_compare['dewpoint'] - df_compare['VC_dew'])
    df_compare['dist_DewPoint_VC'] = np.abs(df_compare['Dew Point (°C)'] - df_compare['VC_dew'])

    # Pour chaque ligne, on détermine quelle mesure est la plus proche des deux autres :
    # - Si 'dewpoint' est plus proche des autres mesures que 'Dew Point (°C)' et 'VC_dew', alors 'dewpoint' est marqué comme plus proche.
    df_compare['dewpoint_closer'] = (df_compare['dist_dewpoint_VC'] < df_compare['dist_dewpoint_DewPoint']) & (df_compare['dist_dewpoint_VC'] < df_compare['dist_DewPoint_VC'])

    # - Si 'Dew Point (°C)' est plus proche des autres mesures que 'dewpoint' et 'VC_dew', alors 'Dew Point (°C)' est marqué comme plus proche.
    df_compare['DewPoint_closer'] = (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_DewPoint_VC'])

    # - Si 'VC_dew' est plus proche des autres mesures que 'dewpoint' et 'Dew Point (°C)', alors 'VC_dew' est marqué comme plus proche.
    df_compare['VC_closer'] = (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_DewPoint'])

    # Calculer les probabilités que chaque mesure soit la plus proche des autres sur l'ensemble des lignes :
    # La probabilité est simplement la proportion de fois où une mesure a été plus proche des autres.
    prob_dewpoint_closer = df_compare['dewpoint_closer'].mean().round(3)
    prob_dewpoint_c_closer = df_compare['DewPoint_closer'].mean().round(3)
    prob_vc_closer = df_compare['VC_closer'].mean().round(3)

    # Afficher les résultats
    # Ces résultats indiquent la probabilité que chaque mesure soit la plus proche des autres sur toutes les lignes de données
    print(f"Probability that 'dewpoint' is closer to the truth: {prob_dewpoint_closer}")
    print(f"Probability that 'Dew Point (°C)' is closer to the truth: {prob_dewpoint_c_closer}")
    print(f"Probability that 'VC_dew' is closer to the truth: {prob_vc_closer}")
    
except Exception as e:
    print(e)

Probability that 'dewpoint' is closer to the truth: 0.829
Probability that 'Dew Point (°C)' is closer to the truth: 0.086
Probability that 'VC_dew' is closer to the truth: 0.078


In [46]:
try:
    df_final['Dew Point (°C)'] = df_final['dewpoint']
    df_final = drop_columns_if_exist(df_final, ['dewpoint'])
except Exception as e:
    print(e)

 Nombre initial de colonnes: 26
Colonne 'dewpoint' Supprimée
 Nombre final de colonnes: 25


In [47]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T°  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [48]:
df_final = add_day_period_and_month(df_final)
print(df_final.columns)

Index(['Wind Direction (°)', 'Wind Gusts (km/h)', 'Wave Height (m)',
       'Average Wave Period (s)', 'Dominant Wave Direction (°)', 'Water T°',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Air T° Height', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitations (mm)', 'Sea Level Pressure (hPa)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'Day Period'],
      dtype='object')


In [49]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T°  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Day Period
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [50]:
display(df_final.info())
display(df_final.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25503 entries, 0 to 25502
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Wind Direction (°)           25503 non-null  float64       
 1   Wind Gusts (km/h)            25503 non-null  float64       
 2   Wave Height (m)              25503 non-null  float64       
 3   Average Wave Period (s)      25503 non-null  float64       
 4   Dominant Wave Direction (°)  25503 non-null  float64       
 5   Water T°                     25503 non-null  float64       
 6   Datetime                     25503 non-null  datetime64[ns]
 7   Lat                          25503 non-null  float64       
 8   Lon                          25503 non-null  float64       
 9   Water Depth (m)              25503 non-null  object        
 10  Station ID                   25503 non-null  object        
 11  Station Zone                 25503 non-nu

None

Index(['Wind Direction (°)', 'Wind Gusts (km/h)', 'Wave Height (m)',
       'Average Wave Period (s)', 'Dominant Wave Direction (°)', 'Water T°',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Air T° Height', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitations (mm)', 'Sea Level Pressure (hPa)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'Day Period'],
      dtype='object')

In [51]:
# Charger les informations de connexion MySQL
path_to_mysql_creds = r"c:\Credentials\mysql_creds.json"
with open(path_to_mysql_creds, 'r') as file:
    content = json.load(file)
    mysql_user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]
    database = "oceanography_data_analysis"

# Connexion à MySQL avec SQLModel
engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{password}@{host}:{port}/{database}", isolation_level='AUTOCOMMIT')


In [52]:
############# STATION DIMENSION ######################################################################################################################
class DimStation(SQLModel, table=True):
    Station_ID: Optional[int] = Field(default=None, primary_key=True)
    Station_Zone: str = Field(index=True, unique=True)
    Lat: float
    Lon: float

    # Relations avec les faits
    meteorological_data: List["FactMeteorologicalData"] = Relationship(back_populates="station")
    oceanographic_data: List["FactOceanographicData"] = Relationship(back_populates="station")


############# DATE DIMENSION ######################################################################################################################
class DimStation(SQLModel, table=True):
    __tablename__ = 'dimstation'  # Nom explicite de la table
    __table_args__ = {'extend_existing': True}  # Permet de réutiliser et modifier la table si elle existe déjà

    Station_ID: Optional[int] = Field(default=None, primary_key=True)
    Station_Zone: str = Field(index=True, unique=True)
    Lat: float
    Lon: float


    # Relations avec les faits
    meteorological_data: List["FactMeteorologicalData"] = Relationship(back_populates="date")
    oceanographic_data: List["FactOceanographicData"] = Relationship(back_populates="date")

############# METEO FACT ######################################################################################################################
class FactMeteorologicalData(SQLModel, table=True):
    Meteo_ID: Optional[int] = Field(default=None, primary_key=True)
    Datetime: datetime

    T_C: Optional[float] = Field(alias="T°(C°)")
    Relative_Humidity: Optional[float] = Field(alias="Relative Humidity (%)")
    Dew_Point: Optional[float] = Field(alias="Dew Point (°C)")
    Precipitations: Optional[float] = Field(alias="Precipitations (mm)")
    Sea_Level_Pressure: Optional[float] = Field(alias="Sea Level Pressure (hPa)")
    Low_Clouds: Optional[float] = Field(alias="Low Clouds (%)")
    Middle_Clouds: Optional[float] = Field(alias="Middle Clouds (%)")
    High_Clouds: Optional[float] = Field(alias="High Clouds (%)")
    Visibility: Optional[float] = Field(alias="Visibility (km)")
    Wind_Speed: Optional[float] = Field(alias="Wind Speed (10m)")
    Wind_Direction: Optional[float] = Field(alias="Wind Direction (°)")
    Wind_Gusts: Optional[float] = Field(alias="Wind Gusts (km/h)")
    Barometer_Elevation: Optional[str]
    Air_T_Height: Optional[str]

    Station_ID: Optional[int] = Field(foreign_key="dimstation.Station_ID")
    Date_ID: Optional[int] = Field(foreign_key="dimdate.Date_ID")

    # Relations avec les dimensions
    station: Optional[DimStation] = Relationship(back_populates="meteorological_data")
    date: Optional[DimDate] = Relationship(back_populates="meteorological_data")

    # Ajout d'une contrainte d'unicité pour les enregistrements météorologiques par combinaison Datetime, Station_ID et Date_ID
    __table_args__ = (UniqueConstraint('Datetime', 'Station_ID', 'Date_ID', name='uix_meteo'),)

############# OCEAN FACT ######################################################################################################################
class FactOceanographicData(SQLModel, table=True):
    Ocean_ID: Optional[int] = Field(default=None, primary_key=True)
    Datetime: datetime

    Wave_Height: Optional[float] = Field(alias="Wave Height (m)")
    Average_Wave_Period: Optional[float] = Field(alias="Average Wave Period (s)")
    Dominant_Wave_Direction: Optional[float] = Field(alias="Dominant Wave Direction (°)")
    Water_Temp: Optional[float] = Field(alias="Water T°")
    Water_Depth: Optional[str] = Field(alias="Water Depth (m)")
    Sea_Temp_Depth: Optional[str] = Field(alias="Sea Temperature Depth (m)")

    Station_ID: Optional[int] = Field(foreign_key="dimstation.Station_ID")
    Date_ID: Optional[int] = Field(foreign_key="dimdate.Date_ID")

    # Relations avec les dimensions
    station: Optional[DimStation] = Relationship(back_populates="oceanographic_data")
    date: Optional[DimDate] = Relationship(back_populates="oceanographic_data")

    # Ajout d'une contrainte d'unicité pour les enregistrements océaniques par combinaison Datetime, Station_ID et Date_ID
    __table_args__ = (UniqueConstraint('Datetime', 'Station_ID', 'Date_ID', name='uix_ocean'),)

# === Créer les tables dans la base de données ===
SQLModel.metadata.create_all(engine)

############# DIMENSION TABLES PREPARATION ######################################################################################################################

# Dimension Station
df_stations_ready_to_load = df_final[['Station ID', 'Station Zone', 'Lat', 'Lon']].drop_duplicates().reset_index(drop=True)
df_stations_ready_to_load['Station_Key'] = df_stations_ready_to_load.index + 1  # Primary key

# Dimension Date
df_final['Year'] = df_final['Datetime'].dt.year
df_final['Month'] = df_final['Datetime'].dt.month
df_final['Day'] = df_final['Datetime'].dt.day
df_final['DayOfWeek'] = df_final['Datetime'].dt.weekday

dates = df_final[['Year', 'Month', 'Day', 'DayOfWeek','Day Period']].drop_duplicates().reset_index(drop=True)
dates['Date_Key'] = dates.index + 1  # Primary key

############# FOREIGN KEYS MAPPING ######################################################################################################################

# Mapping Station_Key dans df_final
station_map = dict(zip(df_stations_ready_to_load['Station ID'], df_stations_ready_to_load['Station_Key']))
df_final['Station_Key'] = df_final['Station ID'].map(station_map)

# Mapping Date_Key dans df_final
date_map = dict(zip(dates[['Year', 'Month', 'Day']].apply(tuple, axis=1), dates['Date_Key']))
df_final['Date_Key'] = df_final.apply(lambda row: date_map.get((row['Year'], row['Month'], row['Day'])), axis=1)

############# FACT TABLES PREPARATIONS ######################################################################################################################

# Table Fact_MeteorologicalData
fact_meteo_cols = ['Datetime', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
                   'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
                   'Visibility (km)', 'Wind Speed (10m)', 'Wind Direction (°)', 'Wind Gusts (km/h)',
                   'Barometer Elevation', 'Air T° Height', 'Station_Key', 'Date_Key']

df_fact_meteo = df_final[fact_meteo_cols].copy()
df_fact_meteo['Meteo_ID'] = df_fact_meteo.index + 1  # Primary key

# Table Fact_OceanographicData
fact_ocean_cols = ['Datetime', 'Wave Height (m)', 'Average Wave Period (s)',
                   'Dominant Wave Direction (°)', 'Water T°', 'Water Depth (m)', 'Sea Temperature Depth (m)',
                   'Station_Key', 'Date_Key']
df_fact_ocean = df_final[fact_ocean_cols].copy()
df_fact_ocean['Ocean_ID'] = df_fact_ocean.index + 1  # Primary key


############# DATA INSERTION ######################################################################################################################

# Insérer les stations dans la table DimStation en évitant les doublons
try:
    with Session(engine) as session:
        for _, row in df_stations_ready_to_load.iterrows():
            # Vérifier si la station existe déjà
            existing_station = session.query(DimStation).filter(DimStation.Station_Zone == row['Station Zone']).first()
            if not existing_station:
                station = DimStation(Station_Zone=row['Station Zone'], Lat=row['Lat'], Lon=row['Lon'])
                session.add(station)
        session.commit()
    print("✅ Stations insérées avec succès dans DimStation.")
except Exception as e:
    print(f"❌ Erreur lors de l'insertion des stations dans DimStation : {e}")

# Insérer les dates dans la table DimDate en évitant les doublons
try:
    with Session(engine) as session:
        for _, row in dates.iterrows():
            # Vérifier si la date existe déjà
            existing_date = session.query(DimDate).filter(DimDate.Year == row['Year'], 
                                                          DimDate.Month == row['Month'], 
                                                          DimDate.Day == row['Day']).first()
            if not existing_date:
                date = DimDate(Year=row['Year'], Month=row['Month'], Day=row['Day'],
                               DayOfWeek=row['DayOfWeek'], Quarter=1, MonthName="January", DayPeriod="00:00:00")
                session.add(date)
        session.commit()
    print("✅ Dates insérées avec succès dans DimDate.")
except Exception as e:
    print(f"❌ Erreur lors de l'insertion des dates dans DimDate : {e}")

# Insérer les données météorologiques dans la table FactMeteorologicalData en évitant les doublons
try:
    with Session(engine) as session:
        for _, row in df_fact_meteo.iterrows():
            # Vérifier si la combinaison Datetime, Station_ID et Date_ID existe déjà
            existing_meteo_data = session.query(FactMeteorologicalData).filter(
                FactMeteorologicalData.Datetime == row['Datetime'],
                FactMeteorologicalData.Station_ID == row['Station_Key'],
                FactMeteorologicalData.Date_ID == row['Date_Key']).first()
            if not existing_meteo_data:
                meteo_data = FactMeteorologicalData(
                    Datetime=row['Datetime'], T_C=row['T°(C°)'], Relative_Humidity=row['Relative Humidity (%)'],
                    Dew_Point=row['Dew Point (°C)'], Precipitations=row['Precipitations (mm)'],
                    Sea_Level_Pressure=row['Sea Level Pressure (hPa)'], Low_Clouds=row['Low Clouds (%)'],
                    Middle_Clouds=row['Middle Clouds (%)'], High_Clouds=row['High Clouds (%)'],
                    Visibility=row['Visibility (km)'], Wind_Speed=row['Wind Speed (10m)'], Wind_Direction=row['Wind Direction (°)'],
                    Wind_Gusts=row['Wind Gusts (km/h)'], Barometer_Elevation=row['Barometer Elevation'],
                    Air_T_Height=row['Air T° Height'], Station_ID=row['Station_Key'], Date_ID=row['Date_Key']
                )
                session.add(meteo_data)
        session.commit()
    print("✅ Données météorologiques insérées avec succès dans FactMeteorologicalData.")
except Exception as e:
    print(f"❌ Erreur lors de l'insertion des données météorologiques dans FactMeteorologicalData : {e}")

# Insérer les données océaniques dans la table FactOceanographicData en évitant les doublons
try:
    with Session(engine) as session:
        for _, row in df_fact_ocean.iterrows():
            # Vérifier si la combinaison Datetime, Station_ID et Date_ID existe déjà
            existing_ocean_data = session.query(FactOceanographicData).filter(
                FactOceanographicData.Datetime == row['Datetime'],
                FactOceanographicData.Station_ID == row['Station_Key'],
                FactOceanographicData.Date_ID == row['Date_Key']).first()
            if not existing_ocean_data:
                ocean_data = FactOceanographicData(
                    Datetime=row['Datetime'], Wave_Height=row['Wave Height (m)'], Average_Wave_Period=row['Average Wave Period (s)'],
                    Dominant_Wave_Direction=row['Dominant Wave Direction (°)'], Water_Temp=row['Water T°'],
                    Water_Depth=row['Water Depth (m)'], Sea_Temp_Depth=row['Sea Temperature Depth (m)'],
                    Station_ID=row['Station_Key'], Date_ID=row['Date_Key']
                )
                session.add(ocean_data)
        session.commit()
    print("✅ Données océaniques insérées avec succès dans FactOceanographicData.")
except Exception as e:
    print(f"❌ Erreur lors de l'insertion des données océaniques dans FactOceanographicData : {e}")


c:\Users\f.gionnane\Documents\Ocean ETL Data Analysis\env\Lib\site-packages\sqlmodel\main.py:641: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.DimStation, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


NameError: name 'DimDate' is not defined

Map Visualization

In [ ]:
# map = folium.Map(location=(centre[0],centre[1]), 
#                  tiles="Esri.WorldImagery", 
#                  zoom_start=2.5, attr="Données fournies par Esri")
# for loc in list_coords:
#     lat = loc[2]
#     lon = loc[3]
#     station_id = loc[0]
#     station_zone = loc[1]
#     folium.Marker(
#     location=[lat, lon],
#     popup=f"Station ID: {station_id}\n\nZone: {station_zone}\nLat: {lat}\n\nLon: {lon}",
#     icon=folium.Icon(icon="cloud"),
# ).add_to(map)

# map

In [ ]:
# Charger le fichier GeoJSON depuis l'URL
url = "https://gist.githubusercontent.com/jrrickard/8755532505a40f3b8317/raw/ecd98849d3a5f4502b773b986254f19af3b8d8fb/oceans.json"
geojson_data = requests.get(url).json()

# Créer une carte avec un fond satellite ESRI
m = folium.Map(location=[0, 0], zoom_start=3, tiles="Esri.WorldImagery")

# Ajouter le GeoJSON (les océans) à la carte
folium.GeoJson(geojson_data, name="Oceans").add_to(m)

# Fonction pour générer des coordonnées aléatoires dans l'océan
def random_ocean_coords():
    lat = random.uniform(-60, 60)  # Latitude dans les eaux de l'océan
    lon = random.uniform(-180, 180)  # Longitude dans les eaux de l'océan
    return lat, lon

# Générer un DataFrame avec des coordonnées, températures, hauteur des vagues, vitesse du vent, etc.
data = {
    "Coordinates": [],
    "Temperature (°C)": [],
    "Wave Height (m)": [],
    "Wind Speed (km/h)": []
}

# Générer 10 marqueurs avec des valeurs aléatoires
for _ in range(10):
    lat, lon = random_ocean_coords()
    
    # Générer des valeurs aléatoires pour la température, la hauteur des vagues et la vitesse du vent
    temperature = random.uniform(15, 30)  # Température entre 15 et 30°C
    wave_height = random.uniform(0.5, 5)  # Hauteur des vagues entre 0.5m et 5m
    wind_speed = random.uniform(10, 50)  # Vitesse du vent entre 10 km/h et 50 km/h
    
    # Ajouter ces valeurs dans le DataFrame
    data["Coordinates"].append((lat, lon))
    data["Temperature (°C)"].append(temperature)
    data["Wave Height (m)"].append(wave_height)
    data["Wind Speed (km/h)"].append(wind_speed)

# Créer un DataFrame pandas avec ces données
df = pd.DataFrame(data)

# Ajouter les marqueurs et cercles à la carte avec des couleurs dépendant de la température
for index, row in df.iterrows():
    lat, lon = row["Coordinates"]
    temperature = row["Temperature (°C)"]
    
    # Déterminer la couleur du cercle en fonction de la température
    if temperature < 20:
        fill_color = "blue"
    elif temperature < 25:
        fill_color = "green"
    else:
        fill_color = "red"
    
    # Ajouter un marqueur
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)
    
    # Ajouter un cercle autour du marqueur
    folium.CircleMarker(
        location=[lat, lon],
        radius=15,  # Rayon du cercle
        color="black",
        fill=True,
        fill_color=fill_color,
        fill_opacity=0.5
    ).add_to(m)

# Ajouter une couche de contrôle pour la carte
folium.LayerControl().add_to(m)

# Sauvegarder la carte dans un fichier HTML
m.save("ocean_markers_map.html")

# Affichage de la carte dans l'environnement interactif (si vous êtes dans un environnement Jupyter)


In [ ]:
m